

---

# **Chapter 9: Communication and Coordination**

## **Opening Context**
Objects in a system rarely exist in isolation. They notify each other of state changes, coordinate complex multi-step workflows, delegate responsibilities through chains of processors, and traverse data structures. When communication is implemented through direct method calls, systems become tightly coupled, brittle to change, and difficult to test. This chapter examines four patterns that manage interaction without creating dependencies: Observer for publish-subscribe notification, Mediator for centralized coordination, Chain of Responsibility for sequential delegation, and Iterator for sequential access to aggregate structures. These patterns transform chaotic point-to-point communication into structured, maintainable interaction topologies.

---

## **9.1 Observer Pattern: The Pub/Sub Mechanism**

### **Intent**
*Define a one-to-many dependency between objects so that when one object changes state, all its dependents are notified and updated automatically.*

### **The Problem**
Consider a weather monitoring system where multiple displays (Current Conditions, Statistics, Forecast) must update when weather data changes:

```typescript
// ❌ TIGHT COUPLING: Subject knows about every specific observer
class WeatherData {
    private currentDisplay: CurrentConditionsDisplay;
    private statisticsDisplay: StatisticsDisplay;
    private forecastDisplay: ForecastDisplay;
    
    constructor(
        current: CurrentConditionsDisplay,
        stats: StatisticsDisplay,
        forecast: ForecastDisplay
    ) {
        this.currentDisplay = current;
        this.statisticsDisplay = stats;
        this.forecastDisplay = forecast;
    }
    
    measurementsChanged(): void {
        const temp = this.getTemperature();
        const humidity = this.getHumidity();
        const pressure = this.getPressure();
        
        // Hard-coded dependencies violate OCP
        this.currentDisplay.update(temp, humidity, pressure);
        this.statisticsDisplay.update(temp, humidity, pressure);
        this.forecastDisplay.update(temp, humidity, pressure);
        // Adding a new display requires modifying this class!
    }
    
    // ... getters
}
```

**Problems**:
- **OCP Violation**: Adding displays requires modifying `WeatherData`.
- **Tight Coupling**: Subject knows concrete observer classes.
- **Inflexibility**: Cannot add/remove observers at runtime.
- **Push vs Pull**: Subject must know what data observers need.

### **The Solution: Subscription Model**

The Observer pattern separates subjects (publishers) from observers (subscribers) through an abstract interface.

```typescript
/**
 * OBSERVER INTERFACE
 * The interface for objects that should be notified of changes 
 * in a subject.
 */
interface Observer {
    update(subject: Subject): void;
    // Alternative: update(data: WeatherData): void for push model
}

/**
 * SUBJECT INTERFACE
 * Knows its observers and provides an interface for attaching 
 * and detaching Observer objects.
 */
interface Subject {
    attach(observer: Observer): void;
    detach(observer: Observer): void;
    notify(): void;
}

/**
 * CONCRETE SUBJECT
 * Stores state of interest to ConcreteObserver objects.
 * Sends notification to observers when state changes.
 */
class WeatherData implements Subject {
    private observers: Observer[] = [];
    private temperature: number = 0;
    private humidity: number = 0;
    private pressure: number = 0;
    
    // Subscription management
    attach(observer: Observer): void {
        const index = this.observers.indexOf(observer);
        if (index === -1) {
            this.observers.push(observer);
            console.log(`Observer attached. Total: ${this.observers.length}`);
        }
    }
    
    detach(observer: Observer): void {
        const index = this.observers.indexOf(observer);
        if (index !== -1) {
            this.observers.splice(index, 1);
            console.log(`Observer detached. Total: ${this.observers.length}`);
        }
    }
    
    // Broadcast to all observers
    notify(): void {
        console.log("\n[Subject] Notifying observers...");
        for (const observer of this.observers) {
            observer.update(this); // Pull model: observers query subject
        }
    }
    
    // State change triggers notification
    setMeasurements(temp: number, humidity: number, pressure: number): void {
        console.log(`\n[WeatherData] New measurements: ${temp}°F, ${humidity}%, ${pressure} inHg`);
        this.temperature = temp;
        this.humidity = humidity;
        this.pressure = pressure;
        this.measurementsChanged();
    }
    
    measurementsChanged(): void {
        this.notify();
    }
    
    // Getters for Pull model (observers query specific data they need)
    getTemperature(): number { return this.temperature; }
    getHumidity(): number { return this.humidity; }
    getPressure(): number { return this.pressure; }
}

/**
 * CONCRETE OBSERVERS
 * Maintain a reference to ConcreteSubject object.
 * Store state that should stay consistent with the subject's.
 * Implement the Observer updating interface.
 */
class CurrentConditionsDisplay implements Observer {
    private subject: Subject;
    
    constructor(subject: Subject) {
        this.subject = subject;
        subject.attach(this);
    }
    
    update(subject: Subject): void {
        if (subject instanceof WeatherData) {
            const temp = subject.getTemperature();
            const humidity = subject.getHumidity();
            console.log(`[CurrentConditions] Display: ${temp}°F, ${humidity}% humidity`);
        }
    }
    
    remove(): void {
        this.subject.detach(this);
    }
}

class StatisticsDisplay implements Observer {
    private temperatures: number[] = [];
    private subject: Subject;
    
    constructor(subject: Subject) {
        this.subject = subject;
        subject.attach(this);
    }
    
    update(subject: Subject): void {
        if (subject instanceof WeatherData) {
            const temp = subject.getTemperature();
            this.temperatures.push(temp);
            const avg = this.temperatures.reduce((a, b) => a + b, 0) / this.temperatures.length;
            const min = Math.min(...this.temperatures);
            const max = Math.max(...this.temperatures);
            console.log(`[Statistics] Avg/Min/Max: ${avg.toFixed(1)}/${min}/${max}`);
        }
    }
}

class ForecastDisplay implements Observer {
    private lastPressure: number = 0;
    private subject: Subject;
    
    constructor(subject: Subject) {
        this.subject = subject;
        subject.attach(this);
    }
    
    update(subject: Subject): void {
        if (subject instanceof WeatherData) {
            const pressure = subject.getPressure();
            let forecast = "";
            
            if (pressure > this.lastPressure) {
                forecast = "Improving weather on the way!";
            } else if (pressure === this.lastPressure) {
                forecast = "More of the same";
            } else {
                forecast = "Watch out for cooler, rainy weather";
            }
            this.lastPressure = pressure;
            console.log(`[Forecast] Prediction: ${forecast}`);
        }
    }
}

// Usage
const weatherStation = new WeatherData();

// Attach observers (could happen at runtime based on user preferences)
const currentDisplay = new CurrentConditionsDisplay(weatherStation);
const statsDisplay = new StatisticsDisplay(weatherStation);
const forecastDisplay = new ForecastDisplay(weatherStation);

// State changes automatically notify observers
weatherStation.setMeasurements(80, 65, 30.4);
weatherStation.setMeasurements(82, 70, 29.2);

// Dynamic detachment
currentDisplay.remove();
weatherStation.setMeasurements(78, 90, 29.2); // Only stats and forecast update
```

**Code Explanation**:

- **Subject** (`Subject`): Maintains a list of observers and provides methods to attach/detach. Defines the notification protocol.
- **Observer** (`Observer`): Interface for objects that should be notified of subject changes.
- **ConcreteSubject** (`WeatherData`): Stores actual state. Calls `notify()` when state changes, triggering `update()` on all observers.
- **ConcreteObservers**: Implement specific reactions to state changes. The example uses the **Pull Model** (observers query subject for data they need), which is more flexible than the **Push Model** (subject sends all data in notification).

**Push vs Pull**:
- **Push**: Subject sends data in `update()` parameters. Efficient but assumes all observers need same data.
- **Pull**: Subject reference passed, observers query specific data. More flexible, allows observers to request only relevant state.

### **Modern Application: EventEmitter (Node.js/Event-Driven Architecture)**

```typescript
// Modern Observer implementation using EventEmitter pattern
import { EventEmitter } from 'events';

// Typed events (TypeScript)
interface WeatherEvents {
    'temperature-change': number;
    'humidity-change': number;
    'alert': string;
}

class TypedEventEmitter {
    private emitter = new EventEmitter();
    
    on<K extends keyof WeatherEvents>(event: K, listener: (data: WeatherEvents[K]) => void): void {
        this.emitter.on(event, listener);
    }
    
    emit<K extends keyof WeatherEvents>(event: K, data: WeatherEvents[K]): void {
        this.emitter.emit(event, data);
    }
    
    off<K extends keyof WeatherEvents>(event: K, listener: (data: WeatherEvents[K]) => void): void {
        this.emitter.off(event, listener);
    }
}

// Usage
const weatherEvents = new TypedEventEmitter();

// Observers subscribe to specific events
weatherEvents.on('temperature-change', (temp) => {
    console.log(`Temperature alert: ${temp}°F`);
});

weatherEvents.on('alert', (message) => {
    console.log(`CRITICAL: ${message}`);
});

// Subject emits events
weatherEvents.emit('temperature-change', 95);
weatherEvents.emit('alert', 'Heat wave warning!');
```

### **Observer in React/Frontend (State Management)**

```typescript
// Redux/Zustand/Context API use Observer pattern internally
class Store<T> {
    private state: T;
    private listeners: Set<(state: T) => void> = new Set();
    
    constructor(initialState: T) {
        this.state = initialState;
    }
    
    getState(): T {
        return this.state;
    }
    
    setState(updater: (prev: T) => T): void {
        this.state = updater(this.state);
        this.emit();
    }
    
    subscribe(listener: (state: T) => void): () => void {
        this.listeners.add(listener);
        // Return unsubscribe function
        return () => this.listeners.delete(listener);
    }
    
    private emit(): void {
        this.listeners.forEach(listener => listener(this.state));
    }
}

// React hook implementation
function useStore<T>(store: Store<T>) {
    const [state, setState] = useState(store.getState());
    
    useEffect(() => {
        return store.subscribe(setState); // Subscribe on mount, unsubscribe on unmount
    }, [store]);
    
    return state;
}
```

### **Memory Leak Warning**
Observers must unsubscribe when destroyed, or subjects hold references preventing garbage collection:

```typescript
// ❌ MEMORY LEAK: Observer never detached
class BadComponent {
    constructor(subject: Subject) {
        subject.attach(this); // But never detaches!
    }
}

// ✅ PROPER LIFECYCLE MANAGEMENT
class GoodComponent {
    private unsubscribe: () => void;
    
    constructor(subject: Subject) {
        const handler = () => { /* ... */ };
        subject.attach(handler);
        this.unsubscribe = () => subject.detach(handler);
    }
    
    destroy(): void {
        this.unsubscribe(); // Clean up
    }
}
```

---

## **9.2 Mediator Pattern: Centralizing Communication**

### **Intent**
*Define an object that encapsulates how a set of objects interact. Mediator promotes loose coupling by keeping objects from referring to each other explicitly, and it lets you vary their interaction independently.*

### **The Problem**
Consider a chat room where users can send messages to specific users or broadcast to all. Without a mediator, each user must maintain references to all other users:

```typescript
// ❌ SPAGHETTI DEPENDENCIES: Every user knows about every other user
class User {
    private name: string;
    private contacts: User[] = []; // Direct references!
    
    constructor(name: string) {
        this.name = name;
    }
    
    addContact(user: User): void {
        this.contacts.push(user);
    }
    
    sendMessage(message: string, recipient: User): void {
        // Direct communication - tight coupling
        if (this.contacts.includes(recipient)) {
            recipient.receiveMessage(message, this);
        }
    }
    
    broadcast(message: string): void {
        // Must iterate through all contacts
        this.contacts.forEach(contact => {
            contact.receiveMessage(message, this);
        });
    }
    
    receiveMessage(message: string, sender: User): void {
        console.log(`${this.name} received from ${sender.name}: ${message}`);
    }
}

// Problems:
// 1. Adding a new user requires updating all existing users' contact lists
// 2. Complex logic (blocking, moderation) must be implemented in every user
// 3. Network topology changes require modifying User class
// 4. Testing requires creating entire network of users
```

### **The Solution: Centralized Coordination**

The Mediator pattern introduces a central hub that handles all communication, decoupling colleagues from each other.

```typescript
/**
 * MEDIATOR INTERFACE
 * Declares the communication interface for colleagues.
 */
interface ChatMediator {
    sendMessage(message: string, sender: Colleague): void;
    sendPrivateMessage(message: string, sender: Colleague, recipient: string): void;
    registerUser(user: Colleague): void;
}

/**
 * COLLEAGUE INTERFACE
 * Components that communicate through the mediator.
 */
abstract class Colleague {
    protected mediator: ChatMediator;
    protected name: string;
    
    constructor(mediator: ChatMediator, name: string) {
        this.mediator = mediator;
        this.name = name;
    }
    
    abstract receiveMessage(message: string, sender: string): void;
    abstract send(message: string): void;
    abstract sendTo(recipient: string, message: string): void;
    
    getName(): string {
        return this.name;
    }
}

/**
 * CONCRETE MEDIATOR
 * Implements the cooperative behavior by coordinating Colleague objects.
 * Knows about all colleagues and manages communication rules.
 */
class ChatRoom implements ChatMediator {
    private users: Map<string, Colleague> = new Map();
    private messageHistory: string[] = [];
    
    registerUser(user: Colleague): void {
        this.users.set(user.getName(), user);
        console.log(`[System] ${user.getName()} joined the chat`);
        this.broadcast(`${user.getName()} has joined the room`, "System");
    }
    
    sendMessage(message: string, sender: Colleague): void {
        // Centralized logic: logging, moderation, filtering
        if (this.containsProfanity(message)) {
            console.log(`[System] Message from ${sender.getName()} blocked for inappropriate content`);
            return;
        }
        
        const logEntry = `[${new Date().toISOString()}] ${sender.getName()}: ${message}`;
        this.messageHistory.push(logEntry);
        
        // Broadcast to all users except sender
        this.users.forEach((user, name) => {
            if (name !== sender.getName()) {
                user.receiveMessage(message, sender.getName());
            }
        });
    }
    
    sendPrivateMessage(message: string, sender: Colleague, recipientName: string): void {
        const recipient = this.users.get(recipientName);
        if (recipient) {
            recipient.receiveMessage(`[Private] ${message}`, sender.getName());
            sender.receiveMessage(`[To ${recipientName}] ${message}`, "You");
        } else {
            sender.receiveMessage(`User ${recipientName} not found`, "System");
        }
    }
    
    private broadcast(message: string, sender: string): void {
        this.users.forEach(user => {
            user.receiveMessage(message, sender);
        });
    }
    
    private containsProfanity(message: string): boolean {
        // Centralized moderation logic
        const bannedWords = ['spam', 'scam'];
        return bannedWords.some(word => message.toLowerCase().includes(word));
    }
    
    getHistory(): string[] {
        return [...this.messageHistory];
    }
}

/**
 * CONCRETE COLLEAGUES
 * Communicate with each other only through the mediator.
 */
class ChatUser extends Colleague {
    constructor(mediator: ChatMediator, name: string) {
        super(mediator, name);
    }
    
    send(message: string): void {
        console.log(`\n${this.name} sends broadcast: "${message}"`);
        this.mediator.sendMessage(message, this);
    }
    
    sendTo(recipient: string, message: string): void {
        console.log(`\n${this.name} whispers to ${recipient}: "${message}"`);
        this.mediator.sendPrivateMessage(message, this, recipient);
    }
    
    receiveMessage(message: string, sender: string): void {
        console.log(`[${this.name}'s screen] ${sender}: ${message}`);
    }
}

// Usage
const chatRoom = new ChatRoom();

const alice = new ChatUser(chatRoom, "Alice");
const bob = new ChatUser(chatRoom, "Bob");
const charlie = new ChatUser(chatRoom, "Charlie");

chatRoom.registerUser(alice);
chatRoom.registerUser(bob);
chatRoom.registerUser(charlie);

alice.send("Hello everyone!");
bob.sendTo("Alice", "Hi Alice, private message");
charlie.send("This is spam!"); // Gets blocked by mediator

// Alice and Bob don't know Charlie exists directly - only through mediator
```

**Code Explanation**:

- **Mediator** (`ChatMediator`): Defines the communication protocol between colleague objects.
- **ConcreteMediator** (`ChatRoom`): Implements the mediation logic. Knows about all colleagues and coordinates their interaction. Centralizes cross-cutting concerns (logging, moderation, routing).
- **Colleague** (`Colleague`): Abstract class for components that communicate through the mediator. Each colleague knows only about the mediator, not other colleagues.
- **ConcreteColleagues** (`ChatUser`): Implement specific behaviors but delegate communication to the mediator.

**Benefits**:
- **Decoupling**: Colleagues don't reference each other directly.
- **Single Responsibility**: Communication logic centralized in Mediator, not spread across colleagues.
- **OCP**: New colleague types can be added without changing existing colleagues (only need to know Mediator interface).

### **Mediator vs Observer**
| Mediator | Observer |
|----------|----------|
| Many-to-many communication | One-to-many communication |
| Centralized routing | Broadcast to all subscribers |
| Complex interaction logic in mediator | Simple notification mechanism |
| Colleagues don't know each other | Observers know about subject |

**Hybrid Approach**: Complex systems often use both—Observer for notification, Mediator for complex routing logic.

### **Real-World Application: Redux/Flux Architecture**

```typescript
// Redux Store acts as Mediator between Actions and Reducers
class ReduxStore {
    private reducers: Map<string, Function> = new Map();
    private state: any = {};
    private subscribers: Function[] = [];
    
    // Register "colleagues" (reducers)
    registerReducer(slice: string, reducer: Function): void {
        this.reducers.set(slice, reducer);
    }
    
    // Mediate action dispatching
    dispatch(action: { type: string; payload?: any }): void {
        console.log(`[Mediator] Processing action: ${action.type}`);
        
        // Route to appropriate reducers
        this.reducers.forEach((reducer, slice) => {
            this.state[slice] = reducer(this.state[slice], action);
        });
        
        // Notify observers
        this.subscribers.forEach(sub => sub(this.state));
    }
    
    getState(): any {
        return this.state;
    }
    
    subscribe(callback: Function): void {
        this.subscribers.push(callback);
    }
}

// Components (Colleagues) don't communicate directly
const store = new ReduxStore();
store.registerReducer('user', (state, action) => { /* ... */ });
store.registerReducer('cart', (state, action) => { /* ... */ });
```

---

## **9.3 Chain of Responsibility: Passing Requests Down the Line**

### **Intent**
*Avoid coupling the sender of a request to its receiver by giving more than one object a chance to handle the request. Chain the receiving objects and pass the request along the chain until an object handles it.*

### **The Problem**
Consider an expense approval system where requests must be approved by managers, directors, or VP based on amount:

```typescript
// ❌ CONDITIONAL CHAIN: Tightly coupled decision logic
class ExpenseApprover {
    approveExpense(amount: number): void {
        if (amount <= 100) {
            console.log("Manager approves $${amount}");
        } else if (amount <= 1000) {
            console.log("Director approves $${amount}");
        } else if (amount <= 10000) {
            console.log("VP approves $${amount}");
        } else {
            console.log("Amount too high, needs board approval");
        }
    }
}

// Problems:
// 1. Hard-coded thresholds and approvers
// 2. Cannot change order or add approvers without modifying class
// 3. Sender knows about all potential handlers
// 4. No flexibility in routing (what if Manager can escalate?)
```

### **The Solution: Handler Chain**

The Chain of Responsibility pattern creates a chain of handler objects, each deciding whether to process the request or pass it to the next handler.

```typescript
/**
 * HANDLER INTERFACE
 * Declares interface for handling requests.
 * Optionally implements the successor link.
 */
abstract class ExpenseHandler {
    protected successor: ExpenseHandler | null = null;
    
    setSuccessor(handler: ExpenseHandler): ExpenseHandler {
        this.successor = handler;
        return handler; // For fluent chaining
    }
    
    handleRequest(amount: number): void {
        if (this.canHandle(amount)) {
            this.processRequest(amount);
        } else if (this.successor) {
            console.log(`${this.getName()} cannot handle $${amount}, passing to next...`);
            this.successor.handleRequest(amount);
        } else {
            console.log(`Request for $${amount} cannot be processed - no handler available`);
        }
    }
    
    protected abstract canHandle(amount: number): boolean;
    protected abstract processRequest(amount: number): void;
    protected abstract getName(): string;
}

/**
 * CONCRETE HANDLERS
 * Handle requests they are responsible for.
 * Can access successor if they cannot handle request.
 */
class Manager extends ExpenseHandler {
    protected getName(): string { return "Manager"; }
    
    protected canHandle(amount: number): boolean {
        return amount <= 100;
    }
    
    protected processRequest(amount: number): void {
        console.log(`[Manager] Approved expense of $${amount}`);
    }
}

class Director extends ExpenseHandler {
    protected getName(): string { return "Director"; }
    
    protected canHandle(amount: number): boolean {
        return amount <= 1000;
    }
    
    protected processRequest(amount: number): void {
        console.log(`[Director] Approved expense of $${amount}`);
    }
}

class VP extends ExpenseHandler {
    protected getName(): string { return "VP"; }
    
    protected canHandle(amount: number): boolean {
        return amount <= 10000;
    }
    
    protected processRequest(amount: number): void {
        console.log(`[VP] Approved expense of $${amount}`);
    }
}

class CEO extends ExpenseHandler {
    protected getName(): string { return "CEO"; }
    
    protected canHandle(amount: number): boolean {
        return true; // Can handle any amount
    }
    
    protected processRequest(amount: number): void {
        console.log(`[CEO] Approved expense of $${amount} (final authority)`);
    }
}

// Usage: Configure chain dynamically
const manager = new Manager();
const director = new Director();
const vp = new VP();
const ceo = new CEO();

// Set up chain: Manager -> Director -> VP -> CEO
manager.setSuccessor(director).setSuccessor(vp).setSuccessor(ceo);

// Client only knows about the first handler (entry point)
const approver = manager; // Could be configured at runtime

// Test various amounts
approver.handleRequest(50);   // Manager handles
approver.handleRequest(500);  // Director handles
approver.handleRequest(5000); // VP handles
approver.handleRequest(50000); // CEO handles

// Can modify chain at runtime
console.log("\n--- Modified Chain (removing VP) ---");
manager.setSuccessor(director);
director.setSuccessor(ceo); // Skip VP
approver.handleRequest(5000); // Now CEO handles it
```

**Code Explanation**:

- **Handler** (`ExpenseHandler`): Defines interface for handling requests. Maintains a reference to the successor (next handler in chain).
- **ConcreteHandlers** (`Manager`, `Director`, etc.): Process requests they can handle. If unable to handle, forward to successor.
- **Client**: Initiates request to first handler in chain. Client has no knowledge of which handler will process the request.

**Chain Configuration**:
- **Linear Chain**: A → B → C → D
- **Tree Structure**: Handlers can have multiple successors (complex routing)
- **Circular Chain**: Rare, but possible for retry logic

### **Advanced: Mutable Chain with Preprocessing**

```typescript
class MiddlewareHandler extends ExpenseHandler {
    private middleware: ((amount: number) => boolean)[] = [];
    
    use(fn: (amount: number) => boolean): void {
        this.middleware.push(fn);
    }
    
    protected canHandle(amount: number): boolean {
        // Run all middleware, if any fail, stop processing
        return this.middleware.every(fn => fn(amount));
    }
    
    protected processRequest(amount: number): void {
        console.log(`[Middleware] Processed $${amount}`);
    }
    
    protected getName(): string { return "Middleware"; }
}

// Usage: Express.js-style middleware chain
const authChain = new MiddlewareHandler();
authChain.use((amount) => {
    console.log("Checking authentication...");
    return true;
});
authChain.use((amount) => {
    console.log("Checking authorization...");
    return amount < 10000; // Reject large amounts
});
```

### **Real-World Applications**

**1. HTTP Request Processing (Express.js/Node.js Middleware)**:
```typescript
// Express middleware is Chain of Responsibility
app.use((req, res, next) => {
    if (req.headers.authorization) {
        next(); // Pass to next handler
    } else {
        res.status(401).send("Unauthorized"); // Handle (break chain)
    }
});

app.use((req, res, next) => {
    // Logging middleware
    console.log(`${req.method} ${req.path}`);
    next();
});

app.get('/data', (req, res) => {
    res.json(data); // Final handler
});
```

**2. Exception Handling**:
```typescript
try {
    riskyOperation();
} catch (e) {
    if (e instanceof ValidationError) {
        handleValidation(e);
    } else if (e instanceof NetworkError) {
        handleNetwork(e);
    } else {
        throw e; // Pass up the chain (to global handler)
    }
}
```

---

## **9.4 Iterator Pattern: Traversing Collections**

### **Intent**
*Provide a way to access the elements of an aggregate object sequentially without exposing its underlying representation.*

### **The Problem**
Different data structures (arrays, linked lists, trees, graphs) require different traversal algorithms. Exposing internal structure violates encapsulation:

```typescript
// ❌ EXPOSING INTERNALS: Client knows about internal array
class Playlist {
    private songs: string[] = [];
    
    // Violation: exposing internal array
    getSongs(): string[] {
        return this.songs; // Client can modify internal state!
    }
    
    // Or client must know traversal algorithm
    getSongsLinkedList(): LinkedListNode {
        return this.head; // Exposes linked list structure
    }
}

// Client code coupled to implementation
const playlist = new Playlist();
const songs = playlist.getSongs();
for (let i = 0; i < songs.length; i++) { // Assumes array!
    console.log(songs[i]);
}
```

### **The Solution: Abstract Traversal**

The Iterator pattern encapsulates traversal logic, providing a uniform interface regardless of underlying structure.

```typescript
/**
 * ITERATOR INTERFACE
 * Defines the interface for accessing and traversing elements.
 */
interface Iterator<T> {
    hasNext(): boolean;
    next(): T;
    current(): T;
    reset(): void;
}

/**
 * AGGREGATE INTERFACE
 * Defines an interface for creating an Iterator object.
 */
interface IterableCollection<T> {
    createIterator(): Iterator<T>;
}

/**
 * CONCRETE ITERATOR
 * Implements the Iterator interface.
 * Keeps track of the current position in the traversal.
 */
class ArrayIterator<T> implements Iterator<T> {
    private collection: T[];
    private position: number = 0;
    
    constructor(collection: T[]) {
        this.collection = collection;
    }
    
    hasNext(): boolean {
        return this.position < this.collection.length;
    }
    
    next(): T {
        if (!this.hasNext()) {
            throw new Error("No more elements");
        }
        return this.collection[this.position++];
    }
    
    current(): T {
        if (this.position >= this.collection.length) {
            throw new Error("Iterator out of bounds");
        }
        return this.collection[this.position];
    }
    
    reset(): void {
        this.position = 0;
    }
}

/**
 * Linked List Iterator
 * Different traversal algorithm, same interface
 */
class LinkedListIterator<T> implements Iterator<T> {
    private currentNode: ListNode<T> | null;
    private head: ListNode<T> | null;
    
    constructor(head: ListNode<T> | null) {
        this.head = head;
        this.currentNode = head;
    }
    
    hasNext(): boolean {
        return this.currentNode !== null;
    }
    
    next(): T {
        if (!this.currentNode) {
            throw new Error("No more elements");
        }
        const value = this.currentNode.value;
        this.currentNode = this.currentNode.next;
        return value;
    }
    
    current(): T {
        if (!this.currentNode) {
            throw new Error("Iterator out of bounds");
        }
        return this.currentNode.value;
    }
    
    reset(): void {
        this.currentNode = this.head;
    }
}

// Support class
class ListNode<T> {
    constructor(
        public value: T,
        public next: ListNode<T> | null = null
    ) {}
}

/**
 * CONCRETE COLLECTIONS
 * Implement the IterableCollection interface.
 */
class Playlist implements IterableCollection<string> {
    private songs: string[] = [];
    
    addSong(song: string): void {
        this.songs.push(song);
    }
    
    createIterator(): Iterator<string> {
        return new ArrayIterator(this.songs);
    }
    
    // Internal structure remains encapsulated
    getCount(): number {
        return this.songs.length;
    }
}

class PlaylistLinkedList implements IterableCollection<string> {
    private head: ListNode<string> | null = null;
    private tail: ListNode<string> | null = null;
    
    addSong(song: string): void {
        const newNode = new ListNode(song);
        if (!this.head) {
            this.head = this.tail = newNode;
        } else {
            this.tail!.next = newNode;
            this.tail = newNode;
        }
    }
    
    createIterator(): Iterator<string> {
        return new LinkedListIterator(this.head);
    }
}

// Client code - works with ANY collection type
function playAllSongs(collection: IterableCollection<string>): void {
    const iterator = collection.createIterator();
    
    console.log("Playing playlist:");
    while (iterator.hasNext()) {
        const song = iterator.next();
        console.log(`  🎵 ${song}`);
    }
    
    // Can traverse again
    iterator.reset();
    console.log(`\nReplay (${iterator.hasNext() ? 'yes' : 'no'})`);
}

// Usage
const playlist1 = new Playlist();
playlist1.addSong("Song A");
playlist1.addSong("Song B");

const playlist2 = new PlaylistLinkedList();
playlist2.addSong("Song X");
playlist2.addSong("Song Y");

// Same client code works for both!
playAllSongs(playlist1);
playAllSongs(playlist2);
```

**Code Explanation**:

- **Iterator** (`Iterator`): Defines the traversal interface (`hasNext`, `next`, `reset`). Decouples traversal algorithm from collection.
- **ConcreteIterator** (`ArrayIterator`, `LinkedListIterator`): Implements specific traversal logic for different data structures.
- **Aggregate** (`IterableCollection`): Factory interface for creating iterators.
- **ConcreteAggregate** (`Playlist`, `PlaylistLinkedList`): Return appropriate iterator for their internal structure.

### **Modern Implementation: JavaScript Iterators**

```typescript
// Modern JS/TS uses Symbol.iterator (built-in Iterator pattern)
class ModernPlaylist implements Iterable<string> {
    private songs: string[] = [];
    
    addSong(song: string): void {
        this.songs.push(song);
    }
    
    // Implement Symbol.iterator for for...of loops
    *[Symbol.iterator](): Iterator<string> {
        for (const song of this.songs) {
            yield song;
        }
    }
    
    // Reverse iterator
    reverseIterator(): Iterator<string> {
        let index = this.songs.length - 1;
        return {
            next: () => {
                if (index >= 0) {
                    return { value: this.songs[index--], done: false };
                }
                return { value: undefined as any, done: true };
            }
        } as Iterator<string>;
    }
}

// Usage with modern syntax
const playlist = new ModernPlaylist();
playlist.addSong("Track 1");
playlist.addSong("Track 2");

for (const song of playlist) {
    console.log(song);
}

// Spread operator uses iterator
const songs = [...playlist];
```

### **Iterator Variations**

**1. Filtering Iterator (Decorator)**:
```typescript
class FilteringIterator<T> implements Iterator<T> {
    private innerIterator: Iterator<T>;
    private predicate: (item: T) => boolean;
    
    constructor(iterator: Iterator<T>, predicate: (item: T) => boolean) {
        this.innerIterator = iterator;
        this.predicate = predicate;
    }
    
    hasNext(): boolean {
        // Look ahead to find next valid item
        while (this.innerIterator.hasNext()) {
            const next = this.innerIterator.current();
            if (this.predicate(next)) return true;
            this.innerIterator.next();
        }
        return false;
    }
    
    next(): T {
        if (!this.hasNext()) throw new Error("No more elements");
        return this.innerIterator.next();
    }
    
    current(): T {
        return this.innerIterator.current();
    }
    
    reset(): void {
        this.innerIterator.reset();
    }
}
```

---

## **Chapter Summary**
In this chapter, we examined four patterns managing object communication:

1.  **Observer**: Establishes publish-subscribe relationships. Subject notifies observers automatically of state changes. Foundational to event-driven architectures and modern frontend frameworks. **Status: Essential for reactive systems and loose coupling.**

2.  **Mediator**: Centralizes complex communication between colleagues. Prevents spaghetti dependencies by routing all interaction through a hub. **Status: Critical for complex coordination scenarios (chat, air traffic control, workflow engines).**

3.  **Chain of Responsibility**: Passes requests along a chain of handlers until one processes it. Decouples sender from receiver and allows dynamic handler configuration. **Status: Fundamental for middleware pipelines and approval workflows.**

4.  **Iterator**: Provides sequential access to aggregate objects without exposing internals. Enables polymorphic traversal of different data structures. **Status: Built into modern languages but important to understand for custom collections.**

**Key Distinctions**:
- **Observer**: One-to-many notification (broadcast).
- **Mediator**: Many-to-many coordination (centralized).
- **Chain of Responsibility**: Sequential processing until handled.
- **Iterator**: Sequential access to collection elements.

---

## **Next Chapter Preview**
**Chapter 10: State Management (State, Memento, Visitor, Interpreter)**

The final chapter of our Behavioral patterns coverage addresses objects that change behavior based on internal state, need to capture and restore state, must perform operations across object structures without modification, or need to evaluate language grammar. We examine the **State** pattern for objects that alter behavior when state changes (cleaner than conditionals), the **Memento** pattern for capturing and restoring object state (undo/redo functionality), the **Visitor** pattern for adding operations to object structures without modifying them, and the **Interpreter** pattern for evaluating sentences in a language. These patterns complete the behavioral toolkit, addressing the most complex interaction scenarios in object-oriented design.